In [236]:
#необходимые импорты (sklearn, numpy, pandas, загрузка датасета load_diabetes)
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
X, y = load_diabetes(return_X_y=True)
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

$$
\text{Итоги всего 2-го модуля.} \\
\text{Имеем матрицу признаков X и вектор таргета Y:} \\
\left(\begin{matrix}
x_{11} & x_{12} & x_{13} \\
x_{21} & x_{22} & x_{23} \\
x_{31} & x_{32} & x_{33}
\end{matrix}\right)
\left(\begin{matrix}
\beta_1 \\
\beta_2 \\
\beta_3
\end{matrix}\right)
= 
\left(\begin{matrix}
y_1 \\
y_2 \\
y_3
\end{matrix}\right) \\
\text{Добавляя вектор весов } \beta_i \text{ мы можем создать определённую } \textit{модель машинного обучения.} \\
\text{Такая модель называется } \textit{линейной.} \\
a(x) = \beta_0 + \beta_1 x_{1} + \beta_2 x_{2} + ... + \beta_n x_n \\
\text{Так же мы имеем определенные метрики или } \textit{функционалы качества.} \\
\text{Например MSE - mean squared error (средне-квадратичная ошибка).} \\
||y - a(x)||^2 = \frac{1}{n}\sum_{i=1}^{n} (y - x_i \beta_i)^2 = \frac{1}{n}((y_1 - x_1 \beta_1)^2 + (y_2 - x_2 \beta_2)^2 + ... + (y_n - x_n \beta_n)^2) \\
\text{Или RMSE - root mean squared error:} \\
\sqrt{MSE} = \sqrt{||y - a(x)||^2} = \sqrt{\frac{1}{n}\sum_{i=1}^{n} (y - x_i \beta_i)^2} \\
\text{Так как нам необхожимо, чтобы данная (или любая другая) метрика была минимальной, мы можем найти коэффициенты (веса), }\\
\textit{в экстремуме этой функций.} \text{(минимуме)} \\
||y - a(x)||^2 \rightarrow min \\
\text{В таком случае наша задача сводится к поиску экстремальных точек, то есть, к } \textit{производной.} \\
\text{Линейная регрессия находит такую точку экстремума (в многомерном пространстве признаков) следующим образом:} \\
Y = X \cdot \beta_M + \varepsilon \\
\beta_M = (X^T \cdot X) ^{-1} \cdot X^T \cdot Y, \\
\beta_M - вектор \ весов  \ модели, \ \varepsilon \text{ - вектор случайных ошибок.} \\
$$

In [237]:
#Реализация ЛИНЕЙНОЙ РЕГРЕССИИ из коробки
model1 = LinearRegression()
model1.fit(X, y)
preds1 = model1.predict(X_test)
no_cross_rmse = np.sqrt(mse(y_test, preds1))

In [238]:
#Ручная реализация ЛИНЕЙНОЙ РЕГРЕССИИ
class LR:
    def __init__(self, fit_intercept = True):
        self.intercept = fit_intercept
        self.error = False
    def fit(self, X, y):
        if self.intercept:
            X = np.column_stack((np.ones(X.shape[0]), X))
        X1 = np.dot(X.T, X)
        if np.linalg.det(X1) == 0:
            print('Нет обратной матрицы, |X| = 0')
            self.error = 1
            return
        X2 = np.linalg.inv(X1)
        X3 = np.dot(X2, X.T)    
        self.B = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.intercept = self.B[-1]
        self.B = self.B[:-1]
    def predict(self, X):
        if self.error == 1:
            print('Признаки мультиколлинеарны.')
            return self.error
        if self.intercept:
            X = np.column_stack((np.ones(X.shape[0]), X))
            self.Beta = np.append(self.B, self.intercept)
        else:
            self.Beta = self.B
        return np.dot(X, self.Beta)

model2 = LR()
model2.fit(X, y)
preds2 = model2.predict(X_test)

In [239]:
#сравнение метрик
print('sklearn RMSE: ', (mse(y_test, preds1)) ** 0.5)
print('наша RSME:    ', (mse(y_test, preds2)) ** 0.5)
print('sklearn коэффициенты:', model1.coef_[0:5], '...')
print('наши коэффициенты:   ', model2.B[0:5], '...')
print('sklearn epsilon:', model1.intercept_)
print('наша epsilon:   ', model2.intercept)

sklearn RMSE:  53.85344583676592
наша RSME:     53.853445836765964
sklearn коэффициенты: [  37.90402135 -241.96436231  542.42875852  347.70384391 -931.48884588] ...
наши коэффициенты:    [ 151.34560454   37.90402135 -241.96436231  542.42875852  347.70384391] ...
sklearn epsilon: 151.34560453985995
наша epsilon:    48.67065743196514


$$\text{Второй способ искать минимум у функционала качества - градиентный спуск.} \\
\textit{Градиент - } \text{это вектор, который показывает направление наискорейшего возрастания функции в конкретной точке.} \\
\text{Определяется как } \textit{вектор производных функции} \text{ по каждой из переменных.} \\
\nabla f = \left(\frac{df}{dx_1},\frac{df}{dx_2},\frac{df}{dx_3},...\right) \\
\text{То есть, если функция - 3-хмерный ланшафт, то градиент в точке будет указывать на крутейший подъем.} \\
\text{Это свойство градиента мы можем использовать для нахождения} \textit{ экстремума функционала качества.} \\
\text{Идея градиентного спуска состоит в том, чтобы "прыгнуть в какую-то точку}X_{нач} \text{ на многомерном пространстве признаков, } \\
\text{а потом начать итеративно двигаться в } \textit{сторону, противоположную градиенту.} \\
X_{1} = X_{нач} - \lambda \cdot \nabla f(X_{нач}) \\
X_{2} = X_{1} - \lambda \cdot \nabla f(X_{1}) \\
... \\
X_{конечная} = X_{n-1} - \lambda \cdot \nabla f(X_{n-1}) \\
\lambda - ускоритель \ шага, \ \nabla f(X_n) \  - \ градиент \  \\
\text{При этом, с каждым шагом } \lambda \cdot \nabla f(X_n) \text{ значение градиента будет всё сильнее и сильнее стремиться к нулю.} \\
\text{Поэтому нам необходимо задать какую-то Эпсилон - окресность } \varepsilon \rightarrow 0 \ ( \varepsilon \sim 0
) \text{ и совершать итерации пока шаг не будет меньше } \varepsilon. \\
\text{Сам градиент можно рассчитать по формуле:} \\
\nabla f'_{\beta_1} = \frac{2}{N} \sum_{i=1}^{N} x_1 (\beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n) \\
\nabla f = (f'_{\beta_1},f'_{\beta_2},f'_{\beta_3},...,f'_{\beta_n})
$$

In [240]:
#Реализация ГРАДИЕНТНОГО СПУСКА из коробки
from sklearn.linear_model import SGDClassifier

gradient_model_sklearn = SGDClassifier(fit_intercept=True, l1_ratio=0.15, max_iter = 10_000, alpha=0.0001)
gradient_model_sklearn.fit(X.copy(), y.copy())
preds_sklearn = gradient_model_sklearn.predict(X_test)

In [241]:
#Ручная реализация ГРАДИЕНТНОГО СПУСКА
class GradientDescent():
    def __init__(self, step=0.0001, epsilon = 0.15):
        self.X = None
        self.y = None
        self.step = step
        self.epsilon = epsilon

    def fit(self, X, y):
        self.X = X.copy()
        self.X = np.c_[self.X, np.ones(self.X.shape[0])]
        self.B = np.ones(X.shape[1] + 1)
        self.y = y.copy()
        prev_mse = self.MSE()
        self._iteration()
        next_mse = self.MSE()
        while abs(next_mse - prev_mse) >= self.epsilon:
            prev_mse = self.MSE()
            self._iteration()
            next_mse = self.MSE()
        return self
        
    def _gradient(self):
        return 2 * (self.X.dot(self.B) - self.y).dot(self.X) / self.X.shape[0]
    
    def MSE(self):
        loss = (self.X.dot(self.B) - self.y) 
        return np.mean(loss**2)
    
    def _iteration(self):
        self.B -= self.step * self._gradient()

    def predict(self, X_train):
        return np.c_[X_train, np.ones(X_train.shape[0])].dot(self.B)
    
gradient_model = GradientDescent()
gradient_model.fit(X, y)
preds_mine = gradient_model.predict(X_test)
print('Мой RMSE в градиентном спуске:    ', np.sqrt(mse(y_test, preds_mine)))
print('Sklearn RMSE в градиентном спуске:', np.sqrt(mse(y_test, preds_sklearn)))

Мой RMSE в градиентном спуске:     72.96613624036382
Sklearn RMSE в градиентном спуске: 81.71634963550935


$$
\textbf{Кросс-валидация} \\
\text{Кросс-валидация - процесс последовательного разделения данных на валидационную и обучающие выборки для подбора оптимальных параметров модели,} \\
\text{предотвращения переобучения и усреднения функционала качества. Алгоритм следующий: } \\
\text{данные делятся на N равных частей, затем N часть используется как валидационная, а остальная - как тренировочная.} \\
\text{Затем мы "смещаемся",  валидируем на N+1, но тренируем на другой и т.д., делам соответственно N раз.} \\
\text{После данный процедуры качество усредняется.} \\
$$

$$\text{cross-validation}_{MSE} =
\begin{bmatrix} 
& \textbf{Валидация} & \text{Трейн} & \text{Трейн} & \text{Трейн} & \text{Трейн} \\
& \text{Трейн} & \textbf{Валидация} & \text{Трейн} & \text{Трейн} & \text{Трейн} \\
& \text{Трейн} & \text{Трейн} & \textbf{Валидация} & \text{Трейн} & \text{Трейн} \\
& \text{Трейн} & \text{Трейн} & \text{Трейн} & \textbf{Валидация} & \text{Трейн} \\
& \text{Трейн} & \text{Трейн} & \text{Трейн} & \text{Трейн} & \textbf{Валидация} \\
\end{bmatrix} =>
\begin{bmatrix}
MSE_1 \\
MSE_2 \\
... \\
MSE_{N-1} \\
MSE_N \\
\end{bmatrix} =
\frac{1}{N}\sum_{i=1}^{N}MSE_i$$ 

In [242]:
#Реализация КРОСС-ВАЛИДАЦИИ из коробки
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

splitter = KFold(n_splits=5, shuffle=True, random_state=22)
model = LinearRegression()

cv_result = cross_validate(model, X, y, cv=splitter, scoring='neg_mean_squared_error', return_train_score=True)

print('Cross-validation test RMSE ', np.sqrt(-np.mean(cv_result['test_score'])))
print('Cross-validation train RMSE', np.sqrt(-np.mean(cv_result['train_score'])))
print('No CV RMSE                 ', no_cross_rmse)

Cross-validation test RMSE  54.57185108354138
Cross-validation train RMSE 53.444451847200455
No CV RMSE                  53.85344583676592


In [243]:
#Ручная реализация КРОСС-ВАЛИДАЦИИ
means_test = []
means_train = []
for train, test in splitter.split(X):
   X_train, X_test = X[train], X[test]
   y_train, y_test = y[train], y[test]

   model = LinearRegression() 
   model.fit(X_train, y_train)
   preds_test = model.predict(X_test)
   preds_train = model.predict(X_train)

   means_test.append(mse(y_test, preds_test) ** 0.5)
   means_train.append(mse(y_train, preds_train) ** 0.5)
print('Cross-validation test RMSE ', np.mean(means_test))
print('Cross-validation train RMSE', np.mean(means_train))

from sklearn.datasets import load_diabetes
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Cross-validation test RMSE  54.51587998454352
Cross-validation train RMSE 53.44102090721621


$$
\textbf{Регуляризация} \\
\text{Регуляризация - изменения признаков, приведение их к небольшим значениям в целях избежать переобучения на train выборке.} \\
\text{Выделяют два вида регуляризации: L1 (lasso) и L2 (ridge).} \\
\textbf{L1}\text{ - в качестве регуляризатора берём сумму всех модулей весов модели.} \\
R(\beta) = \sum_{i=1}^{n} {|\beta_1^2|} = \beta_1^2 + \beta_2^2 + ... \beta_n^2 \\ ||y - X_\beta ||_2^2 + \lambda||\beta||_1 \rightarrow min \\ 
\textbf{L2} \text{ - в качестве регуляризатора используется сумма квадратов весов модели.} \\
R(\beta) = \sum_{i=1}^{n} {\beta_i^2} = \beta_1^2 + \beta_2^2 + ... +\beta_n^2 \\ ||y - X_\beta ||_2^2 + \lambda||\beta||_2^2 \rightarrow min \\
\textbf{Elastic Net}\text{ - совмещённая L1 и L2 регуляризация:} \\
||y - X_\beta||^2 + \lambda||\beta||_2^2 + \lambda||\beta||_1 ъ\\
\text{Аналитическое решение для L2 регуляризации:} \\
\beta = (X^T \cdot X + \lambda I)^{-1} X^T y \\
\text{L1 регуляризация не имеет аналитического решения, ввиду абсолютного значения коэффициентов,} \\
\text{и, как следствие, отсутствия производной для } |x|
\text{ в точке 0} <=> |x|' = \frac{x}{|x|}

$$

In [244]:
#L2 Ridge регуляризация из коробки
from sklearn.linear_model import Ridge
model = Ridge(alpha=0)
model.fit(X, y)
preds = model.predict(X_test)
rmse_sklearn = mse(y_test, preds) ** 0.5

In [245]:
#Ручная реализация L2 регуляризации
class RidgeLR(LR):
    def __init__(self, alpha=0):
        super().__init__()
        self.alpha = alpha
        self.rmse = 0
    def fit(self, X, y):
        X = X = np.c_[X, np.ones(X.shape[0])]
        stage1 = np.dot(X.T, X)
        stage2 = stage1 + self.alpha * np.ones(stage1.shape)
        stage3 = np.linalg.inv(stage2)
        stage4 = stage3.dot(X.T)
        self.B = stage4.dot(y)
    def predict(self, X_test):
        X_test = np.c_[X_test, np.ones(X_test.shape[0])]
        return X_test.dot(self.B)
    
    
ridge_model = RidgeLR()
ridge_model.fit(X, y)
preds = ridge_model.predict(X_test)
rmse_mine = mse(y_test, preds) ** 0.5
print('RMSE нашей ridge модели:   ', rmse_mine)
print('RMSE sklearn ridge модели: ', rmse_sklearn)

RMSE нашей ridge модели:    53.37368921191776
RMSE sklearn ridge модели:  53.37368921191778


In [246]:
#Ручная реализация L1 регуляризации
class LassoGD(GradientDescent):
    def __init__(self, alpha= 0.1, *args, **kwargs):
        super().__init__()
        self.alpha = alpha
    def _gradient(self):
        sign = lambda x: 1 if x >= 0 else -1
        sign_m = np.vectorize(sign)(self.B)
        return 2 * (self.X.dot(self.B) - self.y).dot(self.X) / self.X.shape[0] + self.alpha * sign_m
        

In [247]:
#L1 Lasso регуляризация из коробки
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(alpha=0.1, penalty='l1', max_iter=1000, eta0 = 0.15)
sgd_reg.fit(X, y)
preds1 = sgd_reg.predict(X_test)
rmse1 = mse(y_test, preds1) ** 0.5

gd = LassoGD(alpha = 0.1, epsilon = 0.15)
gd.fit(X, y)
preds2 = gd.predict(X_test)
rmse2 = mse(y_test, preds2) ** 0.5
print('sklearn RMSE :', rmse1)
print('моё     RMSE :', rmse2)

sklearn RMSE : 55.642241522866364
моё     RMSE : 82.19780068628188


In [248]:
#Ручная реализация Elastic Net регуляризации в градиентном спуске
class ElasticNetGD(GradientDescent):
    def __init__(self, alpha = 1, l1_ratio=0.5, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.alpha = alpha
        self.l1_ratio = l1_ratio
    def _gradient(self):
        sign = lambda x: 1 if x >= 0 else -1
        sign_m = np.vectorize(sign)(self.B)
        gradient = (self.X.dot(self.B) - self.y).dot(self.X) / self.X.shape[0]
        gradient -= self.alpha * ((1 - self.l1_ratio) * self.B + self.l1_ratio * sign_m)
        return gradient

In [235]:
#Elastic Net регуляризаця из коробки
from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha = 0.1)
sk_preds = enet.fit(X, y).predict(X_test)
d = enet.get_params()

model = ElasticNetGD(epsilon = 0.01, alpha = d['alpha'], l1_ratio=d['l1_ratio'])
model.fit(X, y)
preds = model.predict(X_test)

print('sklearn RMSE :',mse(y_test, sk_preds)**0.5)
print('моё     RMSE :',mse(y_test, preds)**0.5)


sklearn RMSE : 71.49971078183998
моё     RMSE : 75.64664472847011
